In [1]:
import xarray as xr
from ocf_blosc2 import Blosc2
from tqdm import tqdm
%load_ext autoreload
from torch.utils.data import DataLoader, IterableDataset
%autoreload 2

In [6]:
from dataset import HDF5Dataset
dataset = HDF5Dataset(['/data/processed_data/processed_train_dev.hdf5'], "/data/sat_np/", "/data/weather_np/", True, True, True, True)
data_loader = DataLoader(dataset, batch_size=4, pin_memory=True, num_workers=8, shuffle=False)
print(f"train dataset len: {len(dataset)}")

Opening file /data/processed_data/processed_train_dev.hdf5.
Warming up the dataloader!


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 18777.38it/s]

train dataset len: 100


In [7]:
for (pv_features, hrv_features, nwp, extra, pv_targets) in (pbar := tqdm(data_loader, total=len(data_loader), ascii=True)):
    print(nwp.shape)

 20%|#########################4                                                                                                     | 5/25 [00:04<00:14,  1.38it/s]

torch.Size([4, 6, 10, 128, 128])
torch.Size([4, 6, 10, 128, 128])
torch.Size([4, 6, 10, 128, 128])
torch.Size([4, 6, 10, 128, 128])
torch.Size([4, 6, 10, 128, 128])
torch.Size([4, 6, 10, 128, 128])
torch.Size([4, 6, 10, 128, 128])


 32%|########################################6                                                                                      | 8/25 [00:04<00:06,  2.44it/s]

torch.Size([4, 6, 10, 128, 128])


 40%|##################################################4                                                                           | 10/25 [00:06<00:06,  2.21it/s]

...

torch.Size([4, 6, 10, 128, 128])
torch.Size([4, 6, 10, 128, 128])
torch.Size([4, 6, 10, 128, 128])
torch.Size([4, 6, 10, 128, 128])


 84%|#########################################################################################################8                    | 21/25 [00:07<00:00,  4.51it/s]

torch.Size([4, 6, 10, 128, 128])
torch.Size([4, 6, 10, 128, 128])
torch.Size([4, 6, 10, 128, 128])
torch.Size([4, 6, 10, 128, 128])
torch.Size([4, 6, 10, 128, 128])
torch.Size([4, 6, 10, 128, 128])
torch.Size([4, 6, 10, 128, 128])
torch.Size([4, 6, 10, 128, 128])


100%|##############################################################################################################################| 25/25 [00:09<00:00,  2.69it/s]

torch.Size([4, 6, 10, 128, 128])
